# Assessment 2 - Data Preparation Tasks

## Emma Waterman, Student ID # S00415342

## Task 1

**Instructions**

Good structure of Python Jupyter Notebook.

Containing title cells, subtitle cells.
Python codes are reasonably separated into groups (code cells) with functionalities.
Containing meaningful comments and sensible variable and function names.
Make sure to:

Use the text cell in Google Colab to explain your code and interpret the outputs. Your explanation of the code and interpretation of the outputs is as important as the code itself.
Use different level of headings to make the hierarchy of your submission clear.

**prepare for task 2**

download URL file, save as txt file and open in Excel.

Select all column headers, select 'Format Cells' and select date format - DD-Month(abbreviated)-Year.

Save excel file as .CSV.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Task 2

**Instructions**

Download csv data with pandas with below code:

import pandas as pd
deaths_df=pd.read_csv: 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'Links to an external site.



Note: You may find you have already been provided with the solution for Task 2

In [4]:
import pandas as pd
from pandas import Series, DataFrame

data = pd.read_csv('deaths_global_covid19copy101.csv') #read csv file
data #reads whole file as pandas dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'deaths_global_covid19copy101.csv'

## Task 3

**Instructions**

Display the first five rows of the loaded data (2 marks) and do a short summary of the data (3 marks).

Note: There is a function that can create some descriptive statistics, what is that function? When you attempt this, you may also want to consider whether the statistics you create make sense, e.g. does it make sense to calculate the average of longitudes of countries/provinces/states in this dataset?

In [5]:
import pandas as pd
from pandas import Series, DataFrame


data = pd.read_csv('deaths_global_covid19copy101.csv') #read csv file

data.loc[:4] #slices dataframe to equal first 5 rows. Remember, index starting point = 0.


FileNotFoundError: [Errno 2] No such file or directory: 'deaths_global_covid19copy101.csv'

In [ ]:
data.describe() #provides descriptive statistics for whole csv file.


In [ ]:
grouped_data = data.groupby('Country/Region')[data.columns[50:60]].mean(1) #group data by 'Country/Region'
# Calculate Mean of every column per country = Cumulative average deaths per date by country #sample size columns 50 through to 60.

import numpy as np

filtered_data = grouped_data.replace(0, np.nan) #replaces 0 with 'Not a Number' value

def format_with_scale(val): #formatting function for values
    if pd.isna(val):
        return ' '  #returns blank if value = 0
    elif abs(val) >= 1_000_000:
        return f'{val/1_000_000:.1f}M'  # Millions #returns number of a million with suffix 'M'
    elif abs(val) >= 1_000:
        return f'{val/1_000:.1f}K'      # Thousands #returns number of a thousand with suffix 'k'
    else:
        return f'{val:.1f}'             # Regular numbers #No suffix attached, Round to 1 decimal place


styled_df = filtered_data.style.set_caption("Average Covid-19 Deaths per Country/Region & Date").set_table_styles([
    {'selector': 'th', 'props': [('font-weight', 'bold')]},  #create title, medium font size, Bold fond.
    {'selector': 'caption', 'props': [('font-weight', 'bold'), ('font-size', 'Medium')]}
])

styled_df = styled_df.format(formatter=format_with_scale) #formats table to include title style and scale, rounding to 1 decimal place

styled_df

## Task 4

**Instructions**

Get daily death cases worldwide.

Note: We are after the cumulative worldwide confirmed cases (summarising daily death cases in overall countries). You should produce ONE series that lists the cumulative worldwide confirmed cases against date, i.e. the index is the date and the value is the cumulative worldwide confirmed cases.

In [ ]:
import os
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

data = pd.read_csv('deaths_global_covid19copy101.csv')

date_columns = data.columns[4:]

cumulative_worldwide = data[date_columns].sum()

cumulative_worldwide.index = pd.to_datetime(cumulative_worldwide.index,format= '%d-%b-%y')

monthly = cumulative_worldwide.groupby(cumulative_worldwide.index.to_period('M')).sum()

monthly.index = monthly.index.to_timestamp()

def format_val(x):
    if pd.isna(x): return ' '
    elif x >= 1_000_000: return f'{x/1_000_000:.2f}M'
    elif x >= 1_000: return f'{x/1_000:.2f}K'
    return f'{x:.0f}'


formatted_monthly = monthly.map(format_val)
pd.set_option('display.max_rows', None)  # Show all rows

formatted_monthly




## Task 5

**Instructions**

Get daily increase of death cases via defining a function. Hint: Use the death cases of today minus the death cases of yesterday from the data obtained in Task 4.

Note: This time you are calculating  the daily cases worldwide, that is, the NEW cases confirmed just on a specific day and NOT the cumulative number for each day.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('deaths_global_covid19copy101.csv')

def get_daily_death_increase(data):

    date_cols = data.columns[4:]
    global_total = data[date_cols].sum(axis=0)
    daily_increase = global_total.diff().fillna(0).round(2)
    return daily_increase

daily_increase = get_daily_death_increase(data)

percentage = daily_increase.pct_change()

daily_difference_percentage_change = percentage.fillna(0).round(2).replace([np.inf, -np.inf], 0)

daily_difference_percentage_change

## Task 6

**Instructions**

Visualise the data obtained in Task 4 with library matplotlib.

Choose a diagram/plot that makes sense. Please note that you need to plot the data in Task 4, NOT Task 5. Don’t forget about all the elements in a plot: Have you labelled your x-axis? How about y-axis? Legend?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
from matplotlib.ticker import FuncFormatter

data = pd.read_csv('deaths_global_covid19copy101.csv')

date_columns = data.columns[4:]
worldwide = data[date_columns].sum()
worldwide.index = pd.to_datetime(worldwide.index,format= '%d-%b-%y')

def convert_str(x):
    if isinstance(x, str):
        x = x.upper().replace(',', '')
        if 'M' in x:
            return float(x.replace('M', '')) * 1_000_000
        elif 'K' in x:
            return float(x.replace('K', '')) * 1_000
        else:
            return float(x)
    return x

cumulative_worldwide = worldwide.apply(convert_str)

cumulative_worldwide.index = pd.to_datetime(cumulative_worldwide.index)
cumulative_worldwide = cumulative_worldwide.sort_index()

quarterly_sum = cumulative_worldwide.resample('QE').sum()

plt.figure(figsize=(16,10))

plt.plot(quarterly_sum.index, quarterly_sum.values, marker='o',ms=10, linestyle='-')

quarter_labels = (quarterly_sum.index - pd.offsets.QuarterEnd(startingMonth=3) + pd.offsets.QuarterBegin()).strftime('%b-%Y')

plt.title('Worldwide Covid-19 Deaths Over Time (Quarterly)',fontsize=25.0, fontweight= 'bold', fontname='Century Schoolbook', pad=25)

plt.xlabel('Yearly Quarters', fontsize=20, fontname='Century Schoolbook',labelpad=25)
plt.ylabel('Number of Deaths',fontsize=20, fontname='Century Schoolbook', labelpad=25)

plt.xticks(ticks=quarterly_sum.index, labels=quarter_labels, rotation=45, fontsize=12)


ticks = np.arange(5_000_000, 100_000_001, 5_000_000)
def millions_formatter(x, pos):
    return f'{x/1_000_000:.0f}M'

plt.gca().yaxis.set_major_formatter(FuncFormatter(millions_formatter))

plt.style.use('seaborn-v0_8-pastel')
plt.show()

